In [ ]:
import pandas as pd
import numpy as np

# Convert list of tuples into a DataFrame
frame_1 = [ 
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 
    (0, 0, 0, 0, 0, 0, 1, 0, 0, 1), 
    (0, 0, 0, 0, 0, 1, 1, 1, 1, 1), 
    (0, 0, 0, 0, 1, 1, 1, 1, 1, 1),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 1),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
] 
frame_2 = [ 
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 
    (0, 0, 0, 0, 0, 1, 0, 0, 1, 0), 
    (0, 0, 0, 0, 1, 1, 1, 1, 1, 0), 
    (0, 0, 0, 1, 1, 1, 1, 1, 1, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 1, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
] 
frame_3 = [ 
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 
    (0, 0, 0, 0, 1, 0, 0, 1, 0, 0), 
    (0, 0, 0, 1, 1, 1, 1, 1, 0, 0), 
    (0, 0, 1, 1, 1, 1, 1, 1, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 1, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
] 
frame_4 = [ 
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 
    (0, 0, 0, 1, 0, 0, 1, 0, 0, 0), 
    (0, 0, 1, 1, 1, 1, 1, 0, 0, 0), 
    (0, 1, 1, 1, 1, 1, 1, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 1, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
] 
frame_5 = [ 
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 
    (0, 0, 1, 0, 0, 1, 0, 0, 0, 0), 
    (0, 1, 1, 1, 1, 1, 0, 0, 0, 0), 
    (1, 1, 1, 1, 1, 1, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 1, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
] 

df = pd.DataFrame(data)

# Compute column-wise sums
column_sums = df.sum()

# Find the first column index where sum > 2
valid_indices = [i for i, sum_val in enumerate(column_sums) if sum_val > 1]

if valid_indices:
    leftmost_col_index = min(valid_indices)  # Get the leftmost valid column index
    print(f"Leftmost column index with sum > 2 is: {leftmost_col_index}")
else:
    print("No column has a sum greater than 2.")


In [ ]:
#### Best version: works out left and right_most_col

import os
import pandas as pd
import numpy as np

# Define base directory and input folder
base_dir = "/mnt/d/Flora project/Mask_Test2/Masks"
input_folder = os.path.join(base_dir, "Basic_Masks")

print("CP1")
# Define stripe widths to iterate over
stripe_widths = [2, 4, 8, 16]

# Get all CSV file paths in the "Unaltered" folder
csv_files = []
for root, _, files in os.walk(input_folder):  # Walk through subdirectories
    for file in files:
        if file.endswith(".csv"):
            csv_files.append(os.path.join(root, file))

print(f"Found {len(csv_files)} CSV files.")  # Debugging check

print("CP2")

# Function to find the leftmost column index where sum > 7 and clean invalid 1s
def get_leftmost_col(data):
    column_sums = data.sum()
    
    # Find valid column indices where sum > 7
    valid_indices = [i for i, sum_val in enumerate(column_sums) if sum_val > 7]
    
    if valid_indices:
        leftmost_col_index = min(valid_indices)  # Get the leftmost valid column
        
        # Zero out all 1s in columns before the leftmost valid column
        data.iloc[:, :leftmost_col_index] = data.iloc[:, :leftmost_col_index].replace(1, 0)
        
        return leftmost_col_index
    else:
        return None  # Return None if no valid column exists


# Function to find the rightmost column containing a 1
def get_rightmost_col(data):
    col_indices = np.where(data.eq(1).any(axis=0))[0]  # Get all columns with at least one `1`
    return max(col_indices) if col_indices.size > 0 else None  # Return the rightmost column index


# Function to process each mask
def process_mask(csv_file, stripe_widths):
    # Read CSV without headers
    data = pd.read_csv(csv_file, header=None)

    # Convert to numeric (handle potential issues with non-numeric values)
    data = data.apply(pd.to_numeric, errors='coerce')

    # Extract filename without extension
    file_name = os.path.splitext(os.path.basename(csv_file))[0]
    clip_name = os.path.basename(os.path.dirname(csv_file))

    # Get leftmost column where sum > 7
    leftmost_col_index = get_leftmost_col(data)
    
    if leftmost_col_index is None:
        print(f"Skipping {file_name}: No column with sum > 7 found.")
        return  # Skip processing if no valid column

    # Get rightmost column where any 1 appears
    rightmost_col_index = get_rightmost_col(data)

    if rightmost_col_index is None or rightmost_col_index < leftmost_col_index:
        print(f"Skipping {file_name}: No valid rightmost column found.")
        return  # Skip processing if rightmost column isn't valid

    # Correct fish length to cover all 1s
    fish_length = rightmost_col_index - leftmost_col_index + 1

    print(f"Processing {file_name}: Leftmost = {leftmost_col_index}, Rightmost = {rightmost_col_index}, Fish length = {fish_length}")
    
    print("CP3")
    for stripe_width in stripe_widths:
        data_copy = data.copy()  # Copy original data
        cycle_length = 2 * stripe_width

        # Define stripe pattern
        sequence = np.arange(leftmost_col_index, rightmost_col_index + 1)
        filtered_sequence = sequence[(np.arange(len(sequence)) % cycle_length) < stripe_width]

        # Apply the stripes
        for i in filtered_sequence:
            data_copy.iloc[:, i] = np.where(data_copy.iloc[:, i] > 0, 2, data_copy.iloc[:, i])

        # Define output directory and create if it doesn't exist
        output_dir = os.path.join(base_dir, "Altered_Masks3", clip_name, f"Alt{stripe_width}")
        os.makedirs(output_dir, exist_ok=True)
        print("CP4")

        # Save the modified CSV
        output_file = os.path.join(output_dir, f"{file_name}.csv")
        data_copy.to_csv(output_file, index=False, header=False)

        print(f"Processed {file_name} for Alt{stripe_width}")

# Loop through all CSV files and process them
for csv_file in csv_files:
    process_mask(csv_file, stripe_widths)
    print("CP5")

print("All altered CSV files have been saved.")


In [1]:
##### Code fixed to start the stripes from the correct 'end' (head - left/right) of the fish

import os
import pandas as pd
import numpy as np

# Define base directory and input folder
base_dir = "/mnt/d/Flora project/Mask_Test2/Masks"
input_folder = os.path.join(base_dir, "Basic_Masks")

print("CP1")
# Define stripe widths to iterate over
stripe_widths = [2, 4, 8, 16]

# Get all CSV file paths in the input folder
csv_files = []
for root, _, files in os.walk(input_folder):  
    for file in files:
        if file.endswith(".csv"):
            csv_files.append(os.path.join(root, file))

print(f"Found {len(csv_files)} CSV files.")
print("CP2")

# Function to determine head and tail columns based on movement direction
def get_head_tail_cols(data, direction):
    column_sums = data.sum()
    valid_indices = [i for i, sum_val in enumerate(column_sums) if sum_val > 7]

    col_indices = np.where(data.eq(1).any(axis=0))[0]  # Columns with at least one `1`

    if not valid_indices or not col_indices.size:
        return None, None  # Skip if no valid columns

    if direction == "L":
        head_col = min(valid_indices)
        tail_col = max(col_indices)
    elif direction == "R":
        head_col = max(valid_indices)
        tail_col = min(col_indices)
    else:
        return None, None  # Unknown direction

    return head_col, tail_col

# Function to process each mask
def process_mask(csv_file, stripe_widths):
    # Read CSV without headers
    data = pd.read_csv(csv_file, header=None)
    data = data.apply(pd.to_numeric, errors='coerce')  # Ensure numeric values

    # Extract filename and clip name
    file_name = os.path.splitext(os.path.basename(csv_file))[0]
    clip_name = os.path.basename(os.path.dirname(csv_file))

    # Determine movement direction
    if "_L_" in clip_name:
        direction = "L"
    elif "_R_" in clip_name:
        direction = "R"
    else:
        print(f"Skipping {file_name}: Unknown movement direction.")
        return

    # Get head and tail columns
    head_col, tail_col = get_head_tail_cols(data, direction)

    if head_col is None or tail_col is None or tail_col < head_col:
        print(f"Skipping {file_name}: No valid columns found.")
        return

    # Calculate fish length
    fish_length = abs(tail_col - head_col) + 1

    print(f"Processing {file_name}: Direction = {direction}, Head = {head_col}, Tail = {tail_col}, Fish Length = {fish_length}")

    print("CP3")
    for stripe_width in stripe_widths:
        data_copy = data.copy()
        cycle_length = 2 * stripe_width

        # Define stripe pattern
        sequence = np.arange(head_col, tail_col + 1)
        filtered_sequence = sequence[(np.arange(len(sequence)) % cycle_length) < stripe_width]

        # Apply the stripes
        for i in filtered_sequence:
            data_copy.iloc[:, i] = np.where(data_copy.iloc[:, i] > 0, 2, data_copy.iloc[:, i])

        # Define output directory
        output_dir = os.path.join(base_dir, "Altered_Masks", clip_name, f"Alt{stripe_width}")
        os.makedirs(output_dir, exist_ok=True)
        print("CP4")

        # Save the modified CSV
        output_file = os.path.join(output_dir, f"{file_name}.csv")
        data_copy.to_csv(output_file, index=False, header=False)

        print(f"Processed {file_name} for Alt{stripe_width}")

# Loop through all CSV files and process them
for csv_file in csv_files:
    process_mask(csv_file, stripe_widths)
    print("CP5")

print("All altered CSV files have been saved.")


CP1
Found 446 CSV files.
CP2
Processing Botia_lohachata_1_C_L_3_L_frame_0_mask: Direction = L, Head = 1398, Tail = 1736, Fish Length = 339
CP3
CP4
Processed Botia_lohachata_1_C_L_3_L_frame_0_mask for Alt2
CP4
Processed Botia_lohachata_1_C_L_3_L_frame_0_mask for Alt4
CP4
Processed Botia_lohachata_1_C_L_3_L_frame_0_mask for Alt8
CP4
Processed Botia_lohachata_1_C_L_3_L_frame_0_mask for Alt16
CP5
Processing Botia_lohachata_1_C_L_3_L_frame_10_mask: Direction = L, Head = 1369, Tail = 1705, Fish Length = 337
CP3
CP4
Processed Botia_lohachata_1_C_L_3_L_frame_10_mask for Alt2
CP4
Processed Botia_lohachata_1_C_L_3_L_frame_10_mask for Alt4
CP4
Processed Botia_lohachata_1_C_L_3_L_frame_10_mask for Alt8
CP4
Processed Botia_lohachata_1_C_L_3_L_frame_10_mask for Alt16
CP5
Processing Botia_lohachata_1_C_L_3_L_frame_11_mask: Direction = L, Head = 1366, Tail = 1705, Fish Length = 340
CP3
CP4
Processed Botia_lohachata_1_C_L_3_L_frame_11_mask for Alt2
CP4
Processed Botia_lohachata_1_C_L_3_L_frame_11_mask 